In [2]:
%matplotlib inline
import random
from mxnet import autograd, np, npx
from d2l import mxnet as d2l

npx.set_np()

# Exercises week 2
## 3.1.6.2

We want to optimise the cost function: <br>
$J(w) = \parallel y - Xw \parallel^2$ <br>
Partial derivative with respect to $w$: <br>
$\frac{\partial J(w)}{\partial w} = X^T(y - Xw)$ <br>
Set this to 0 and solve:
\begin{eqnarray*}
    X^T(y-Xw) &=& 0 \\
    w &=& (X^TX)^{-1}X^Ty
\end{eqnarray*}

Since this is a closed formula, it will work better than gradient descent if we can computationally handle the size of matrix $X$. If this is not the case, stochastic gradient descent (because of minibatching) works better.

## 3.2.9.1

If the squared loss function is used, initialising the weights to zero would only change the number of epochs needed to reach convergence. Since the squared loss function is convex, it only has one critical point, which means that the initial values of the weights does not determine whether that critical point will be reached, only when.

## 3.2.9.6



In [17]:
def synthetic_data(w, b, num_examples):  #@save
    """Generate y = Xw + b + noise."""
    X = np.random.normal(0, 1, (num_examples, len(w)))
    y = np.dot(X, w) + b
    y += np.random.normal(0, 0.01, y.shape)
    return X, y.reshape((-1, 1))

true_w = np.array([2, -3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)

def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    # The examples are read at random, in no particular order
    random.shuffle(indices)
    for i in range(0, num_examples, batch_size):
        batch_indices = np.array(indices[i:min(i + batch_size, num_examples)])
        yield features[batch_indices], labels[batch_indices]

w = np.random.normal(0, 0.01, (2, 1))
b = np.zeros(1)
w.attach_grad()
b.attach_grad()

def linreg(X, w, b):  #@save
    """The linear regression model."""
    return np.dot(X, w) + b

def squared_loss(y_hat, y):  #@save
    """Squared loss."""
    return (y_hat - y.reshape(y_hat.shape))**2 / 2

def sgd(params, lr, batch_size):  #@save
    """Minibatch stochastic gradient descent."""
    for param in params:
        param[:] = param - lr * param.grad / batch_size

lr = 0.1
num_epochs = 3
net = linreg
loss = squared_loss
batch_size = 10

for epoch in range(num_epochs):
    for X, y in data_iter(batch_size, features, labels):
        with autograd.record():
            l = loss(net(X, w, b), y)  # Minibatch loss in `X` and `y`
        # Because `l` has a shape (`batch_size`, 1) and is not a scalar
        # variable, the elements in `l` are added together to obtain a new
        # variable, on which gradients with respect to [`w`, `b`] are computed
        l.backward()
        sgd([w, b], lr, batch_size)  # Update parameters using their gradient
    train_l = loss(net(features, w, b), labels)
    print(f'epoch {epoch + 1}, loss {float(train_l.mean()):f}')

print(f'error in estimating w: {true_w - w.reshape(true_w.shape)}')
print(f'error in estimating b: {true_b - b}')

epoch 1, loss 0.000051
epoch 2, loss 0.000051
epoch 3, loss 0.000050
error in estimating w: [-0.0007329 -0.0006671]
error in estimating b: [0.00031614]


For learning rate 0.03: <br>
epoch 1, loss 0.037290
epoch 2, loss 0.000132
epoch 3, loss 0.000048
error in estimating w: [ 0.00096571 -0.00061107]
error in estimating b: [0.00026894]

For learning rate 0.01: <br>
epoch 1, loss 2.212858
epoch 2, loss 0.294010
epoch 3, loss 0.039279
error in estimating w: [ 0.11729062 -0.17064309]
error in estimating b: [0.18917513]

For learning rate 0.05: <br>
epoch 1, loss 0.000650
epoch 2, loss 0.000047
epoch 3, loss 0.000048
error in estimating w: [6.9332123e-04 1.5974045e-05]
error in estimating b: [-0.00017262]

For learning rate 0.1: <br>
epoch 1, loss 0.000051
epoch 2, loss 0.000051
epoch 3, loss 0.000050
error in estimating w: [-0.0007329 -0.0006671]
error in estimating b: [0.00031614]

It seems that a higher learning rate will increase the rate of decrease for the loss function significantly, even with small changes, as can be seen in the change from 0.01 to 0.03. If the learning rate is very high (here: 0.1), the change in loss is minimal.

## 3.6.9.1

In [7]:
import sys

def softmax(X):
    X_exp = np.exp(X)
    partition = X_exp.sum(1, keepdims=True)
    return X_exp / partition

X = np.random.normal(0, 1, (2, 5))
X_prob = softmax(X)
X_prob, X_prob.sum (1)

print(sys.getsizeof(np.exp(50)))

32


We now see that the size of exp(50) in memory is 32 bytes. If the arrays become large, this means that we will get very large result arrays, which take up a lot of space in memory. Since large arrays and lots of iterations are necessary to perform regression, this might become a problem.


## 3.6.9.4

In some cases it is probably better to return a list of labels, ordered by likelihood. So for, as stated in the exercise, if we have a medical diagnosis, we could return a list of possible diagnosis, each labeled by how likely they are. This will allow the system to help a doctor make an informed diagnosis, by combining the output of the system and their own expertise.